In [1]:
from collections import defaultdict
import os
import sys
import pickle
import random
import requests
import time
import tqdm

from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
from pytorch_ranger import Ranger
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.utils.data as td
from torch.utils.tensorboard import SummaryWriter

sys.path.append("/Users/xuxiaoan/Downloads/recsys-rl-master")
from utils import (EvalDataset, OUNoise, Prioritized_Buffer, get_beta, 
                   preprocess_data, to_np, hit_metric, dcg_metric)

In [2]:
data_dir = "data"
rating = "ratings.csv"

params = {
    'max_epoch': 1,
    'batch_size': 64,
    'embedding_dim': 8,
    'hidden_dim': 16,
    'N': 5, # memory size for state_repr
    'ou_noise':False,
    'value_lr': 1e-4,
    'value_decay': 1e-4,
    'policy_lr': 1e-4,
    'policy_decay': 1e-6,
    'state_repr_lr': 1e-4,
    'state_repr_decay': 1e-3,
    'log_dir': 'logs/final/',
    'gamma': 0.8,
    'min_value': -10,
    'max_value': 10,
    'soft_tau': 1e-3,
    'buffer_size': 100000
}

## 0. Problem statement

Traditional recommendation task can be treated as sequental desicion making problem.
Recommender (i.e. agent) interacts with users (i.e. environment) to sequentally suggest set of items.
The goal is to maximize clients' satisfaction (i.e. reward).
More specifically:
- State is a vector $a \in R^{3\cdot embedding\_dim}$ computed using the user embedding and the embeddings of `N` latest positive interactions. In the code (replay buffer) state is represented  by `(user, memory)`
- Action is a vector $a \in R^{embedding\_dim}$. To get ranking score we took dot product of
the action and the item embedding (similar to word2vec and other embedding models).
- Reward is taken from user-item matrix (1 if rating > 3, 0 otherwise)

Reinforcement Learning can help recommendation at least in 2 ways.
1. User’s preference on previous items will affect his choice on the next items. 
User tends to give a higher rating if he has consecutively received more satisfied items (and vice versa). 
So, it would be more reasonable to model the recommendation as a sequential decision making process.
2. It is important to use long-term planning in recommendations. For example, after reading the weather forecast, the user is not willing
to read similar news. On the other hand, after watching funny videos or reading memes the user can constanly do the same.

In [3]:
# Movielens (1M) data from the https://github.com/hexiangnan/neural_collaborative_filtering
if not os.path.isdir('./data'):
    os.mkdir('./data')
    
file_path = os.path.join(data_dir, rating)
if os.path.exists(file_path):
    print("Skip loading " + file_path)
else:
    with open(file_path, "wb") as tf:
        print("Load " + file_path)
        r = requests.get("https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/" + rating)
        tf.write(r.content)
        
(train_data, train_matrix, valid_data, valid_matrix, test_data, test_matrix,
 user_num, item_num, appropriate_users, mapping, reverse_mapping) = preprocess_data(data_dir, rating)

Skip loading data/ratings.csv


## 1. Environment

- **Observation space**. As mentioned before, to get state we need `N` latest positive items (`memory`) and embedding of user. `State_Repr_Module` transform it to the vector of dimensionality `embedding_dim * 3`.

- **Action space**. For every user we sample nonrelated items (the same count as related). All `available_items` which wasn't viewed before form action space.

Given a state we get action embedding, compute dot product between this embedding and embeddings of all items in action space, take 1 top ranked item, compute reward, update `viewed_items` and memory, and store transition in buffer.

In [4]:
class Env():
    def __init__(self, user_item_matrix):
        self.matrix = user_item_matrix
        self.item_count = item_num
        self.memory = np.ones([user_num, params['N']]) * item_num
        # memory is initialized as [item_num] * N for each user
        # it is padding indexes in state_repr and will result in zero embeddings

    # 候选集合为x个有评分的items和x和无评分的items，推荐item个数等同于rating>3的item个数
    def reset(self, user_id):
        self.user_id = user_id
        self.viewed_items = []
        self.related_items = np.argwhere(self.matrix[self.user_id] > 0)[:, 1]
        self.good_items = np.argwhere(self.matrix[self.user_id] > 3)[:, 1]
        self.num_rele = len(self.related_items)
        self.num_good = len(self.good_items)

        # self.available_items = np.arange(9066)
        self.nonrelated_items = np.random.choice(
            list(set(range(self.item_count)) - set(self.related_items)), self.num_rele)

        self.available_items = np.zeros(self.num_rele*2)
        self.available_items[::2] = self.related_items
        self.available_items[1::2] = self.nonrelated_items

        return torch.tensor([self.user_id]), torch.tensor(self.memory[[self.user_id], :])
    
    def step(self, action, action_emb=None, buffer=None):
        initial_user = self.user_id
        initial_memory = self.memory[[initial_user], :]

        # 通过rating计算reward（按照原文）
        if to_np(action) not in self.related_items:
            rating = 0
            reward = 0
        else:
            rating = self.matrix[self.user_id, to_np(action)[0]]
            reward = (rating - 3) / 2

        self.viewed_items.append(to_np(action)[0])

        # rating>3才更新状态
        if reward >0:
            if len(action) == 1:
                self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]
            else:
                self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action[0]]

        # 推荐商品个数 = rating>3的商品个数
        if len(self.viewed_items) == self.num_good:
            done = 1
        else:
            done = 0

        # 向replay buffer存储SARS
        if buffer is not None:
            buffer.push(np.array([initial_user]), np.array(initial_memory), to_np(action_emb)[0], 
                        np.array([reward]), np.array([self.user_id]), self.memory[[self.user_id], :], np.array([done]))

        return torch.tensor([self.user_id]), torch.tensor(self.memory[[self.user_id], :]), reward, rating, done

## 2. Model

### Overall model

<img src="img/full_model.png" width="500" height="350">

In [5]:
class Actor_DRR(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
    
        self.layers = nn.Sequential(
            nn.Linear(embedding_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        self.initialize()

    def initialize(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)

    def forward(self, state):
        return self.layers(state)

    # 只在available items中选择最接近的item，否则大量reward=0，不利于训练
    def get_action(self, user, memory, state_repr, 
                   action_emb,
                   items=torch.tensor([i for i in range(item_num)]),
                   return_scores=False
                  ):
        # state没有作用？
        state = state_repr(user, memory)
        scores = torch.bmm(state_repr.item_embeddings(items).unsqueeze(0), 
                         action_emb.T.unsqueeze(0)).squeeze(0)
        if return_scores:
            return scores, torch.gather(items, 0, scores.argmax(0))
        else:
            return torch.gather(items, 0, scores.argmax(0))

In [6]:
class Critic_DRR(nn.Module):
    def __init__(self, state_repr_dim, action_emb_dim, hidden_dim):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(state_repr_dim + action_emb_dim, 1)
        )

        self.initialize()
        
    def initialize(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = self.layers(x)
        return x

### State representation

<img src="img/state_representation.png" width="350" height="250">

In [7]:
class State_Repr_Module(nn.Module):
    def __init__(self, user_num, item_num, embedding_dim, hidden_dim):
        super().__init__()
        self.user_embeddings = nn.Embedding(user_num, embedding_dim)
        # item需要padding补齐
        self.item_embeddings = nn.Embedding(item_num+1, embedding_dim, padding_idx=int(item_num))
        self.drr_ave = torch.nn.Conv1d(in_channels=params['N'], out_channels=1, kernel_size=1)
        
        self.initialize()
            
    def initialize(self):
        nn.init.normal_(self.user_embeddings.weight, std=0.01)
        nn.init.normal_(self.item_embeddings.weight, std=0.01)
        self.item_embeddings.weight.data[-1].zero_()
        nn.init.uniform_(self.drr_ave.weight)
        self.drr_ave.bias.data.zero_()

    def forward(self, user, memory):
        user_embedding = self.user_embeddings(user.long())

        item_embeddings = self.item_embeddings(memory.long())
        drr_ave = self.drr_ave(item_embeddings).squeeze(1)
        
        return torch.cat((user_embedding, user_embedding * drr_ave, drr_ave), 1)

For evaluation we take 1 positive and 99 sampled negatives items per batch, select 10 items with best scores and calculate hit_rate@10 and nDCG@10.
During training we choose user 6039 and track `hit` and `dcg` only for him (for evaluation speed). Final scores was computed on the whole test data.

In [8]:
# load pre-trained memory
with open('logs/memory.pickle', 'rb') as f:
    test_memory = pickle.load(f)

In [9]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    return 0

def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0
    return dcg_at_k(r, k) / dcg_max

In [10]:
def count_greater_than_x(list, x):
    count = 0
    for element in list:
        if element > x:
            count += 1
    return count

In [11]:
def run_evaluation(net, state_representation, training_env_memory):
    test_env = Env(test_matrix)
    test_env.memory = training_env_memory.copy()
    ndcg_at_10_list = []
    precision_at_10_list = []
    for user in range(user_num):
        user, memory = test_env.reset(user)
        if test_env.num_rele < 10:
            continue
        else:
            rating_list = []
            for t in range(10):
                action_emb = net(state_repr(user, memory))
                scores, action = net.get_action(
                    user,
                    torch.tensor(test_env.memory[to_np(user).astype(int), :]),
                    state_representation,
                    action_emb,
                    torch.tensor([item for item in test_env.related_items
                    if item not in test_env.viewed_items]).long(),
                    return_scores=True
                )
                user, memory, reward, rating, done = test_env.step(action)
                rating_list.append(rating)
            ndcg_at_10 = ndcg_at_k(rating_list,10)
            ndcg_at_10_list.append(ndcg_at_10)
            precision_at_10_list.append(count_greater_than_x(rating_list,3)/10)
    return np.mean(precision_at_10_list), np.mean(ndcg_at_10_list)

In [12]:
from collections import Counter
Counter(test_matrix.tocsr().getnnz(1)>=3)

Counter({True: 780, False: 163})

## 3. Training

In [28]:
user, memory = train_env.reset(669)
state = state_repr(user, memory)
get_action(state, value_net)

(tensor(332),
 tensor([[ 0.1853,  0.2085, -0.1968,  0.2282,  0.1742,  0.0438,  0.1719,  0.0227]]))

In [23]:
def get_action(state, net):
    Q_list = []
    with torch.no_grad():
        candidate_items = torch.tensor(
                [item for item in train_env.available_items
                if item not in train_env.viewed_items]
            ).long()
        action_embs = state_repr.item_embeddings(candidate_items)
        for action_emb in action_embs:
            Q_list.append(net(state,action_emb.unsqueeze(0))[0][0])
    max_index = Q_list.index(max(Q_list))
    index = candidate_items[max_index]
    return index,state_repr.item_embeddings(index).detach().unsqueeze(0)

In [13]:
torch.manual_seed(1)

state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
value_net  = Critic_DRR(params['embedding_dim'] * 3, params['embedding_dim'], params['hidden_dim'])
replay_buffer = Prioritized_Buffer(params['buffer_size'])

target_value_net  = Critic_DRR(params['embedding_dim'] * 3, params['embedding_dim'], params['hidden_dim'])
target_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_criterion  = nn.MSELoss()
value_optimizer  = Ranger(value_net.parameters(),  lr=params['value_lr'], 
                          weight_decay=params['value_decay'])
policy_optimizer = Ranger(policy_net.parameters(), lr=params['policy_lr'], 
                          weight_decay=params['policy_decay'])
state_repr_optimizer = Ranger(state_repr.parameters(), lr=params['state_repr_lr'], 
                              weight_decay=params['state_repr_decay'])

writer = SummaryWriter(log_dir=params['log_dir'])

In [14]:
def ddpg_update(training_env,
                step=0,
                batch_size=params['batch_size'], 
                gamma=params['gamma'],
                min_value=params['min_value'],
                max_value=params['max_value'],
                soft_tau=params['soft_tau'],
               ):
    beta = get_beta(step)
    user, memory, action, reward, next_user, next_memory, done = replay_buffer.sample(batch_size, beta)
    user        = torch.FloatTensor(user)
    memory      = torch.FloatTensor(memory)
    action      = torch.FloatTensor(action)
    reward      = torch.FloatTensor(reward)
    next_user   = torch.FloatTensor(next_user)
    next_memory = torch.FloatTensor(next_memory)
    done = torch.FloatTensor(done)
    
    state       = state_repr(user, memory)
    policy_loss = value_net(state, policy_net(state))
    policy_loss = -policy_loss.mean()
    
    next_state     = state_repr(next_user, next_memory)
    next_action    = target_policy_net(next_state)
    target_value   = target_value_net(next_state, next_action.detach())
    expected_value = reward + (1.0 - done) * gamma * target_value
    expected_value = torch.clamp(expected_value, min_value, max_value)

    value = value_net(state, action)
    value_loss = value_criterion(value, expected_value.detach())
    
    state_repr_optimizer.zero_grad()
    policy_optimizer.zero_grad()
    policy_loss.backward(retain_graph=True)
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward(retain_graph=True)
    value_optimizer.step()
    state_repr_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
                target_param.data.copy_(
                    target_param.data * (1.0 - soft_tau) + param.data * soft_tau
                )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

    # writer.add_histogram('value', value, step)
    # writer.add_histogram('target_value', target_value, step)
    # writer.add_histogram('expected_value', expected_value, step)
    # writer.add_histogram('policy_loss', policy_loss, step)

In [15]:
np.random.seed(15)
train_env = Env(train_matrix)
precisions, ndcgs = [], []
step, best_step = 0, 0
users = np.random.permutation(appropriate_users)
ou_noise = OUNoise(params['embedding_dim'], decay_period=10)

for epoch in range(params["max_epoch"]):
    for u in tqdm.tqdm(users):
        user, memory = train_env.reset(u)
        if params['ou_noise']:
            ou_noise.reset()
        for t in range(train_env.num_good):
            action_emb = policy_net(state_repr(user, memory))
            if params['ou_noise']:
                action_emb = ou_noise.get_action(action_emb.detach().cpu().numpy()[0], t)
            action = policy_net.get_action(
                user,
                torch.tensor(train_env.memory[to_np(user).astype(int), :]),
                state_repr,
                action_emb,
                torch.tensor(
                    [item for item in train_env.available_items
                    if item not in train_env.viewed_items]
                ).long()
            )

            # 返回S'R，并将transition传入replay buffer
            user, memory, reward, rating, done = train_env.step(
                action,
                action_emb,
                buffer=replay_buffer
            )

            if len(replay_buffer) > params['batch_size']:
                ddpg_update(train_env, step=step)

            # 每1000个step保存最优参数（使得ndcg@10最大）（使用同一个test_memory）
            if step % 1000 == 0 and step > 0:
                precision, ndcg = run_evaluation(policy_net, state_repr, train_env.memory)
                # writer.add_scalar('precision', precision, step)
                # writer.add_scalar('ndcg', ndcg, step)
                precisions.append(precision)
                ndcgs.append(ndcg)
                print(precision,ndcg)
                if np.mean(np.array([ndcg]) - np.array(ndcgs[best_step])) > 0:
                    # best_step是当前step在ndcgs中的index，ndcgs的元素为从1000开始
                    best_step = step // 1000 - 1
                    torch.save(policy_net.state_dict(), params['log_dir'] + 'best_policy_net.pth')
                    torch.save(value_net.state_dict(), params['log_dir'] + 'best_value_net.pth')
                    torch.save(state_repr.state_dict(), params['log_dir'] + 'best_state_repr.pth')
            step += 1

  0%|          | 0/474 [00:00<?, ?it/s][W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
/var/folders/0w/5fjhh0vx7rbg6mmk_ld1nfvr0000gn/T/ipykernel_21671/3694430785.py:45: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]
/var/folders/0w/5fjhh0vx7rbg6mmk_ld1nfvr0000gn/T/ipykernel_21671/4070442190.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525474122/work/torch/csrc/utils/tensor_new.cpp:233.)
  action      = torch.Floa

0.5571428571428572 0.9183162658071427


 15%|█▍        | 69/474 [00:11<02:41,  2.50it/s]

0.5539083557951483 0.9083497793239135


 22%|██▏       | 102/474 [00:16<02:59,  2.08it/s]

...

 87%|████████▋ | 412/474 [01:09<00:18,  3.39it/s]

0.5905660377358491 0.9308571338852926


 95%|█████████▍| 448/474 [01:14<00:09,  2.74it/s]

0.595956873315364 0.9336544514424988


100%|██████████| 474/474 [01:17<00:00,  6.14it/s]


In [25]:
torch.save(policy_net.state_dict(), params['log_dir'] + 'policy_net_final.pth')
torch.save(value_net.state_dict(), params['log_dir'] + 'value_net_final.pth')
torch.save(state_repr.state_dict(), params['log_dir'] + 'state_repr_final.pth')

In [16]:
# we need memory for validation, so it's better to save it and not wait next time 
with open('logs/memory.pickle', 'wb') as f:
    pickle.dump(train_env.memory, f)
    
with open('logs/memory.pickle', 'rb') as f:
    memory = pickle.load(f)

In [81]:
# load pre-trained memory
with open('logs/memory2.pickle', 'rb') as f:
    memory = pickle.load(f)

## 4. Results

Weights and logs are stored in [this folder](https://drive.google.com/drive/folders/1hsGjh8oHN4uyCmp_wtAyVPTR76ylVVgH?usp=sharing)

In [17]:
no_ou_state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
no_ou_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
no_ou_state_repr.load_state_dict(torch.load('logs/final/' + 'best_state_repr.pth'))
no_ou_policy_net.load_state_dict(torch.load('logs/final/' + 'best_policy_net.pth'))

hit, dcg = run_evaluation(no_ou_policy_net, no_ou_state_repr, train_env.memory)
print('hit rate: ', hit, 'dcg: ', dcg)

/var/folders/0w/5fjhh0vx7rbg6mmk_ld1nfvr0000gn/T/ipykernel_21671/3694430785.py:45: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]


hit rate:  0.592722371967655 dcg:  0.9362027842132209


In [17]:
ou_state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
ou_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
ou_state_repr.load_state_dict(torch.load('logs/ou_noise_04/' + 'best_state_repr.pth'))
ou_policy_net.load_state_dict(torch.load('logs/ou_noise_04/' + 'best_policy_net.pth'))

hit, dcg = run_evaluation(ou_policy_net, ou_state_repr, test_memory)
print('hit rate: ', hit, 'dcg: ', dcg)

/var/folders/0w/5fjhh0vx7rbg6mmk_ld1nfvr0000gn/T/ipykernel_33216/4137504441.py:33: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]


hit rate:  0.502304510193194 dcg:  0.2798106994373345


### Example of trained agents behaviour

Let's choose random user

In [19]:
random_user = np.random.randint(user_num)
print(random_user)

5983


In [23]:
movies = pd.read_csv('/Users/xuxiaoan/Downloads/recsys-rl-master/data/movies.dat', sep='::', header=None, engine='python', names=['id', 'name', 'genre'], encoding="latin-1")
# in the code numeration starts with 0
display(movies[movies['id'].isin(np.argwhere(test_matrix[random_user] > 0)[:, 1] + 1)])

,id,name,genre
44,45,To Die For (1995),Comedy|Drama
64,65,Bio-Dome (1996),Comedy
131,133,Nueba Yol (1995),Comedy|Drama
639,644,Happy Weekend (1996),Comedy
706,715,"Horseman on the Roof, The (Hussard sur le toit...",Drama
979,991,Michael Collins (1996),Drama|War
989,1002,Ed's Next Move (1996),Comedy
1222,1242,Glory (1989),Action|Drama|War


For example we can recommend "Nixon" and "Love Serenade" and see next 3 predictions.

In [26]:
predictions = []

for model, state_representation in zip([ou_policy_net, no_ou_policy_net], [ou_state_repr, no_ou_state_repr]):
    example_env = Env(test_matrix)
    user, memory = example_env.reset(random_user)

    user, memory, reward, _ = example_env.step(torch.tensor([44])) # 13
    user, memory, reward, _ = example_env.step(torch.tensor([1001])) # 1584
    preds = []
    for _ in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user, 
            torch.tensor(example_env.memory[to_np(user).astype(int), :]), 
            state_representation, 
            action_emb,
            torch.tensor(
               [item for item in example_env.available_items
               if item not in example_env.viewed_items]
            ).long()
        )
        user, memory, reward, _ = example_env.step(action)
        preds.append(action)

    predictions.append(preds)

print(predictions[0])
print(predictions[1])

[tensor([132]), tensor([714]), tensor([64])]
[tensor([132]), tensor([64]), tensor([714])]


/var/folders/0w/5fjhh0vx7rbg6mmk_ld1nfvr0000gn/T/ipykernel_33216/4137504441.py:33: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]


Model trained with OU noise recommended related `Comedy` and `Documentary` after that switch recommendations to nonrelated `Crime|Film-Noir|Thriller`.
Model trained without OU noise recommended `Comedy|Drama`, `Children's|Comedy`, `Drama` (two of them are related).

Both models seems to be reasonable.

### Training process logs

<img src=img/learning_curve.png>

Logs are consistent with expectations. Adding noise increase metrics (std=0.4 performs the best, after 0.6 model starts to degrade).